In [ ]:
!pip install SPARQLWrapper

     |████████████████████████████████| 235kB 9.9MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 


In [ ]:
import pandas as pd
import ast
import math
from textblob import TextBlob
import re
import numpy as np
import csv
# import seaborn
import statistics
from SPARQLWrapper import SPARQLWrapper, JSON
import json
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
root_dir = '/content/drive/MyDrive/ProgettoAI/dataset/'
trump = pd.read_csv(root_dir+'trump_corrected.csv')


In [ ]:
class DbpediaObject:

  def __init__(self, string):
    json_parsed = ast.literal_eval(string)
    self.confidence = float(json_parsed['@confidence'])
    self.text = json_parsed['@text']
    self.entities = []
    try:
      self._set_entities(json_parsed['Resources'])
    except:
      None



  def _set_entities(self, lista):
    for el in lista:
      if(el['@surfaceForm'] != 't.co'):
        self.entities.append(Entity(el))


In [ ]:
class Entity:
  def __init__(self, json_parsed):
    self.original_text = json_parsed['@surfaceForm']
    self.label = json_parsed['@URI'].split('/')[-1]
    self.uri = json_parsed['@URI']
    self.similarity = json_parsed['@similarityScore']
    self.types = []
    self.schema = []
    self._set_types(json_parsed['@types'])

  def _set_types(self, raw_list):
      raw_split = raw_list.split(',')
      for el in raw_split:
        x = el.split(':')
        if x[0] == 'DBpedia':
          self.types.append(x[1])
        if x[0] == 'Wikidata':
          self.wikidata = x[1]
        if x[0] == 'Schema':
          self.schema.append(x[1])




In [ ]:
def count_empty_list(lista):
  count = 0
  for el in lista:
    if len(el) == 0:
      count = count +1
  return count

In [ ]:
def entita_filter_by_type(tipo, df_entita):
  cit = list()
  count_cit = list()
  entities_cit = list()
  plain_text = list()

  for index, row in df_entita.iterrows():
    if tipo in row['entita'].types:
      cit.append(row['entita'].label)
      count_cit.append(row['count'])
      entities_cit.append(row['entita'].uri)
      plain_text.append(row['plain_text'])
  return pd.DataFrame(list(zip(cit,count_cit,entities_cit,plain_text)), columns=['label', 'count', 'uri','plain_text']).sort_values('count', ascending=False)


#TRUMP

In [ ]:
lista = []
for el in trump['dbpedia']:
  try:
    lista.append(DbpediaObject(el))
  except:
    print(el)
trump['dbpedia_object'] = lista

In [ ]:
trump=trump.drop('Unnamed: 0', axis=1)
trump=trump.drop('Unnamed: 0.1', axis=1)
trump

,handle,text,is_retweet,original_author,time,entities,extended_entities,dbpedia,dbpedia_object
0,realDonaldTrump,Join me for a 3pm rally - tomorrow at the Mid-...,False,NaN,2016-09-27T22:13:24,"{'user_mentions': [], 'symbols': [], 'urls': [...",NaN,{'@text': 'Join me for a 3pm rally - tomorrow ...,<__main__.DbpediaObject object at 0x7f38af2e7ac8>
1,realDonaldTrump,"Once again, we will have a government of, by a...",False,NaN,2016-09-27T21:08:22,{'media': [{'display_url': 'pic.twitter.com/uY...,{'media': [{'display_url': 'pic.twitter.com/uY...,"{'@text': 'Once again, we will have a governme...",<__main__.DbpediaObject object at 0x7f38af308048>
2,realDonaldTrump,"On National #VoterRegistrationDay, make sure y...",True,GOP,2016-09-27T20:31:14,"{'user_mentions': [{'id_str': '11134252', 'nam...",NaN,"{'@text': ""On National #VoterRegistrationDay, ...",<__main__.DbpediaObject object at 0x7f38af2e7c88>
3,realDonaldTrump,Hillary Clinton's Campaign Continues To Make F...,False,NaN,2016-09-27T20:14:33,"{'user_mentions': [], 'symbols': [], 'urls': [...",NaN,"{'@text': ""Hillary Clinton's Campaign Continue...",<__main__.DbpediaObject object at 0x7f38af2e7c18>
4,realDonaldTrump,"'CNBC, Time magazine online polls say Donald T...",False,NaN,2016-09-27T20:06:25,"{'user_mentions': [{'id_str': '14662354', 'nam...",NaN,"{'@text': ""'CNBC, Time magazine online polls s...",<__main__.DbpediaObject object at 0x7f38af2e7e80>
...,...,...,...,...,...,...,...,...,...
3213,realDonaldTrump,"""@lilredfrmkokomo: @realDonaldTrump My Faceboo...",False,NaN,2016-01-05T03:47:14,"{'user_mentions': [{'id_str': '26122621', 'nam...",NaN,"{'@text': '""@lilredfrmkokomo: @realDonaldTrump...",<__main__.DbpediaObject object at 0x7f38aead7a20>
3214,realDonaldTrump,"""@marybnall01: @realDonaldTrump watched lowell...",False,NaN,2016-01-05T03:44:17,"{'user_mentions': [{'id_str': '3477455725', 'n...",NaN,"{'@text': '""@marybnall01: @realDonaldTrump wat...",<__main__.DbpediaObject object at 0x7f38aeae3208>
3215,realDonaldTrump,"""@ghosthunter_lol: Iowa key endorsement for @r...",False,NaN,2016-01-05T03:42:10,{'media': [{'display_url': 'pic.twitter.com/JB...,{'media': [{'display_url': 'pic.twitter.com/JB...,"{'@text': '""@ghosthunter_lol: Iowa key endorse...",<__main__.DbpediaObject object at 0x7f38aead7908>
3216,realDonaldTrump,"""@iLoveiDevices: @EdwinRo47796972 @happyjack22...",False,NaN,2016-01-05T03:39:11,"{'user_mentions': [{'id_str': '42568997', 'nam...",NaN,"{'@text': '""@iLoveiDevices: @EdwinRo47796972 @...",<__main__.DbpediaObject object at 0x7f38aead7cf8>


In [ ]:
trump_list_entities = list(x.entities for x in trump['dbpedia_object'])


In [ ]:
x = round(count_empty_list(trump_list_entities)/len(trump_list_entities)*100,2)
print("% of tweet without entities: ", x)

% of tweet without entities:  9.88


In [ ]:
num_entities = [i.label for x in trump_list_entities for i in x]
print("Different entities found", len(set(num_entities)))

Different entities found 1856


In [ ]:
num_entities = [i.original_text for x in trump_list_entities for i in x]
print("Different labels found", len(set(num_entities)))

Different labels found 2094


In [ ]:
num_entities = [i for x in trump_list_entities for i in x]
print("Entities found", len(num_entities))

Entities found 7470


In [ ]:
entities_count_trump = dict((el.label,el) for el in [i for x in trump_list_entities for i in x])
entita_trump = list(entities_count_trump.values())
entities_count_trump = list(np.zeros(len(entita_trump), dtype=int))
trump_labels = [x.label for x in entita_trump]
trump_types =  [x.types for x in entita_trump]
# trump_schema =  [x.schema for x in entita_trump]

for el in trump['dbpedia_object']:
  for entity in el.entities:
    index = trump_labels.index(entity.label)
    entities_count_trump[index] = entities_count_trump[index] + 1

entities_frame = pd.DataFrame(list(zip(entita_trump, trump_labels,entities_count_trump,trump_types)), columns=['entita', 'label', 'count','types'])
# entities_frame.sort_values('count', ascending=False)

In [ ]:
trump_types = list(set(tipo for el in entities_frame['types'] for tipo in el))
trump_types.append('Without_type')
trump_types_count = list(np.zeros(len(trump_types),dtype=int))

for ent_list in trump_list_entities:
  for ent in ent_list:
    if not ent.types:
      trump_types_count[len(trump_types_count)-1] = trump_types_count[len(trump_types_count)-1] + 1
    for tipo in ent.types:
      index = trump_types.index(tipo)
      trump_types_count[index] = trump_types_count[index] + 1
tipi_trump_frame = pd.DataFrame(list(zip(trump_types, trump_types_count)), columns = ['tipo', 'count'])
tipi_trump_frame.sort_values('count', ascending=False).iloc[0:20]

,tipo,count
174,Without_type,2817
46,Agent,2241
13,Place,1389
109,Location,1389
100,Person,1353
6,PopulatedPlace,1353
52,Politician,1009
59,Organisation,846
48,Work,831
121,AdministrativeRegion,721


In [ ]:
# total = []
# for en in entities_frame['entita']:
  # partial = []
  # for i in range(0,len(trump)):
    # if en.label in [x.label for x in trump.iloc[i]['dbpedia_object'].entities]:
      # partial.append(i)
  # total.append(partial)

In [ ]:
# with open("trump_entities_occurrence.csv", "w", newline="") as f:
    # writer = csv.writer(f)
    # writer.writerows(total)

In [ ]:
total = []
with open('/content/drive/MyDrive/ProgettoAI/dataset/trump_entities_occurrence.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
      total.append({int(x) for x in row})
         

In [ ]:
entities_frame['occurrences'] = total

In [ ]:
plain_text = []
for index, el in entities_frame.iterrows():
  partial = set()
  for i in el['occurrences']:
    for entity in trump.iloc[i]['dbpedia_object'].entities:
      if entity.label == el['label']:
        partial.add(entity.original_text)
  plain_text.append(partial)


In [ ]:
entities_frame['plain_text'] = plain_text
entities_frame.sort_values('count', ascending=False)

,entita,label,count,types,occurrences,plain_text
40,<__main__.Entity object at 0x7f38aeaed860>,Donald_Trump_on_social_media,330,[],"{2057, 2058, 3082, 1036, 2572, 526, 2574, 1040...","{@RealDonaldTrump, @realDonaldTrump, @realdona..."
9,<__main__.Entity object at 0x7f38aeaed240>,Donald_Trump,298,"[Person, Agent, Politician]","{4, 517, 1544, 2056, 2059, 13, 1550, 16, 1040,...","{DONALD TRUMP, Donald J Trump, Trump, Donald T..."
15,<__main__.Entity object at 0x7f38aead5dd8>,United_States,185,"[PopulatedPlace, Place, Location, Country]","{3072, 516, 6, 10, 1550, 2064, 3094, 1047, 104...","{United States, Union, American, USA, America,..."
4,<__main__.Entity object at 0x7f38aeb2e320>,Hillary_Clinton,173,"[Person, Agent, Politician]","{3, 1547, 13, 1551, 2579, 20, 21, 532, 534, 54...","{HILLARY CLINTON, Clinton, Hillary Clinton, HRC}"
11,<__main__.Entity object at 0x7f38aeaedac8>,Make_America_Great_Again,150,[],"{1536, 1025, 1537, 1538, 4, 2562, 6, 8, 9, 256...","{MAGA, MAKE AMERICA GREAT AGAIN, Make America ..."
...,...,...,...,...,...,...
869,<__main__.Entity object at 0x7f38af010748>,South_Dakota,1,"[Region, PopulatedPlace, Place, Location, Admi...",{1137},{South Dakota}
865,<__main__.Entity object at 0x7f38af028860>,Highness,1,[],{1131},{Highness}
864,<__main__.Entity object at 0x7f38af028630>,Gone_(Dwight_Yoakam_album),1,"[Work, MusicalWork, Album]",{1130},{Gone}
863,<__main__.Entity object at 0x7f38af0102b0>,Prejudice,1,[],{1129},{bigot}


In [ ]:
for i in tipi_trump_frame.sort_values('count', ascending=False).iloc[1:]['tipo']:
  print('-------------------------------','TIPO:',i,'--------------------------------------')
  display(entita_filter_by_type(i,entities_frame).iloc[:20])

------------------------------- TIPO: Agent --------------------------------------


,label,count,uri,plain_text
2,Donald_Trump,298,http://dbpedia.org/resource/Donald_Trump,"{Donald Trump, President Trump, DONALD TRUMP, ..."
0,Hillary_Clinton,173,http://dbpedia.org/resource/Hillary_Clinton,"{Hillary Clinton, Clinton, HRC, HILLARY CLINTON}"
230,Jervy_Cruz,115,http://dbpedia.org/resource/Jervy_Cruz,"{Cruz, CRUZ}"
4,CNN,108,http://dbpedia.org/resource/CNN,"{CNN, cnn}"
31,Fox_News,97,http://dbpedia.org/resource/Fox_News,"{foxnews, Fox News, Fox News Channel, FoxNews,..."
6,Enjoy_Records,89,http://dbpedia.org/resource/Enjoy_Records,"{enjoy!, ENJOY!, Enjoy, Enjoy!}"
119,Ted_Cruz,81,http://dbpedia.org/resource/Ted_Cruz,"{TED CRUZ, Ted Cruz}"
11,Barack_Obama,71,http://dbpedia.org/resource/Barack_Obama,"{Barrack Obama, Obama, President Obama, debating}"
215,Marco_Rubio,70,http://dbpedia.org/resource/Marco_Rubio,"{RUBIO, Marco Rubio, Rubio}"
42,Republican_Party_(United_States),56,http://dbpedia.org/resource/Republican_Party_(...,"{SR, Republican party, Republicans, Republican..."


------------------------------- TIPO: Location --------------------------------------


,label,count,uri,plain_text
2,United_States,185,http://dbpedia.org/resource/United_States,"{USA, U.S., U.S.A., US, United States, America..."
40,Iowa,69,http://dbpedia.org/resource/Iowa,"{IA, Iowa}"
12,Ohio,56,http://dbpedia.org/resource/Ohio,"{Ohio, OHIO}"
76,New_Hampshire,48,http://dbpedia.org/resource/New_Hampshire,"{NH, New Hampshire}"
43,South_Carolina,43,http://dbpedia.org/resource/South_Carolina,"{SC, South Carolina, SOUTH CAROLINA}"
21,Florida,42,http://dbpedia.org/resource/Florida,{Florida}
57,Indiana,40,http://dbpedia.org/resource/Indiana,"{IN, Indiana, INDIANA}"
23,New_York_City,40,http://dbpedia.org/resource/New_York_City,"{NewYork, New York, NYC, NY, New York City}"
64,Wisconsin,31,http://dbpedia.org/resource/Wisconsin,"{WISCONSIN, WI, Wisconsin}"
35,Pennsylvania,30,http://dbpedia.org/resource/Pennsylvania,"{PA, Pennsylvania}"


------------------------------- TIPO: Place --------------------------------------


...

,label,count,uri,plain_text
0,Beck_(manga),1,http://dbpedia.org/resource/Beck_(manga),{BECK}


------------------------------- TIPO: Pope --------------------------------------


,label,count,uri,plain_text
0,Pope_Francis,1,http://dbpedia.org/resource/Pope_Francis,{Pope Francis}


------------------------------- TIPO: Racecourse --------------------------------------


,label,count,uri,plain_text
0,Churchill_Downs,1,http://dbpedia.org/resource/Churchill_Downs,"{Louisville, Kentucky}"


### Sentiment


In [ ]:
sentiments = trump['text'].apply(TextBlob)

In [ ]:
polarities = [x.sentiment.polarity for x in sentiments]
subjectivities = [x.sentiment.subjectivity for x in sentiments]

In [ ]:
trump['sentiment'] = polarities
trump['subjectivity'] = subjectivities
trump

,handle,text,is_retweet,original_author,time,entities,extended_entities,dbpedia,dbpedia_object,sentiment,subjectivity
0,realDonaldTrump,Join me for a 3pm rally - tomorrow at the Mid-...,False,NaN,2016-09-27T22:13:24,"{'user_mentions': [], 'symbols': [], 'urls': [...",NaN,{'@text': 'Join me for a 3pm rally - tomorrow ...,<__main__.DbpediaObject object at 0x7f38af2e7ac8>,-0.125000,0.100000
1,realDonaldTrump,"Once again, we will have a government of, by a...",False,NaN,2016-09-27T21:08:22,{'media': [{'display_url': 'pic.twitter.com/uY...,{'media': [{'display_url': 'pic.twitter.com/uY...,"{'@text': 'Once again, we will have a governme...",<__main__.DbpediaObject object at 0x7f38af308048>,0.000000,0.000000
2,realDonaldTrump,"On National #VoterRegistrationDay, make sure y...",True,GOP,2016-09-27T20:31:14,"{'user_mentions': [{'id_str': '11134252', 'nam...",NaN,"{'@text': ""On National #VoterRegistrationDay, ...",<__main__.DbpediaObject object at 0x7f38af2e7c88>,0.500000,0.888889
3,realDonaldTrump,Hillary Clinton's Campaign Continues To Make F...,False,NaN,2016-09-27T20:14:33,"{'user_mentions': [], 'symbols': [], 'urls': [...",NaN,"{'@text': ""Hillary Clinton's Campaign Continue...",<__main__.DbpediaObject object at 0x7f38af2e7c18>,-0.400000,0.600000
4,realDonaldTrump,"'CNBC, Time magazine online polls say Donald T...",False,NaN,2016-09-27T20:06:25,"{'user_mentions': [{'id_str': '14662354', 'nam...",NaN,"{'@text': ""'CNBC, Time magazine online polls s...",<__main__.DbpediaObject object at 0x7f38af2e7e80>,0.250000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...
3213,realDonaldTrump,"""@lilredfrmkokomo: @realDonaldTrump My Faceboo...",False,NaN,2016-01-05T03:47:14,"{'user_mentions': [{'id_str': '26122621', 'nam...",NaN,"{'@text': '""@lilredfrmkokomo: @realDonaldTrump...",<__main__.DbpediaObject object at 0x7f38aead7a20>,1.000000,0.750000
3214,realDonaldTrump,"""@marybnall01: @realDonaldTrump watched lowell...",False,NaN,2016-01-05T03:44:17,"{'user_mentions': [{'id_str': '3477455725', 'n...",NaN,"{'@text': '""@marybnall01: @realDonaldTrump wat...",<__main__.DbpediaObject object at 0x7f38aeae3208>,0.933333,0.833333
3215,realDonaldTrump,"""@ghosthunter_lol: Iowa key endorsement for @r...",False,NaN,2016-01-05T03:42:10,{'media': [{'display_url': 'pic.twitter.com/JB...,{'media': [{'display_url': 'pic.twitter.com/JB...,"{'@text': '""@ghosthunter_lol: Iowa key endorse...",<__main__.DbpediaObject object at 0x7f38aead7908>,0.000000,1.000000
3216,realDonaldTrump,"""@iLoveiDevices: @EdwinRo47796972 @happyjack22...",False,NaN,2016-01-05T03:39:11,"{'user_mentions': [{'id_str': '42568997', 'nam...",NaN,"{'@text': '""@iLoveiDevices: @EdwinRo47796972 @...",<__main__.DbpediaObject object at 0x7f38aead7cf8>,0.000000,0.000000


In [ ]:
polarity_mean = []
polarity_median = []
for index, el in entities_frame.iterrows():
  partial = list()
  for i in el['occurrences']:
    partial.append(trump.iloc[i]['sentiment'])
  polarity_mean.append(statistics.mean(partial))
  polarity_median.append(statistics.median(partial))


In [ ]:
entities_frame['sentiment_mean'] = polarity_mean
entities_frame['sentiment_median'] = polarity_median

In [ ]:
entities_frame[entities_frame['label']=='Vladimir_Putin']

,entita,label,count,types,occurrences,plain_text,sentiment_mean,sentiment_median
247,<__main__.Entity object at 0x7f38aeb4ee10>,Vladimir_Putin,7,"[Person, Agent, Politician]","{546, 227, 2626, 677, 518, 2971, 638}",{Putin},-0.182309,-0.257727


In [ ]:
# entities_frame.to_csv('entities_trump.csv')

# Tabella entità usate in comune

In [ ]:
# partial_matrix = np.zeros([len(entities_frame), len(entities_frame)], dtype=int)
# for i in range(0,len(entities_frame)):
  # print(i, end= ' ')
  # for j in range(i+1,len(entities_frame)):
    # partial_matrix[i,j] = len(entities_frame.iloc[i]['occurrences'].intersection(entities_frame.iloc[j]['occurrences']))

In [ ]:
# np.savetxt("trump_entities_matrix.csv", partial_matrix, delimiter=",")

In [ ]:
trump_entities_matrix = np.genfromtxt('/content/drive/MyDrive/ProgettoAI/dataset/trump_entities_matrix.csv', delimiter=',')

# Query sparql



In [ ]:
# per utilizzarla inserire all'interno della query il token "TEXT_HERE" dove si vuole inserire l'uri dell'entità
def party_from_uri(uri_list, query, variable):
  tot = []
  for uri in uri_list:
    # print(uri)
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query.replace('TEXT_HERE', uri))   
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    # tot.append(results["results"]["bindings"])
    if len(results["results"]["bindings"]) != 0:
      party = results["results"]["bindings"][0][variable]
      res_entity = {
              "res" :uri,
              variable: party["value"],
          }
    else:
      res_entity = {
              "res" :uri,
              variable: None,
          }
    tot.append(res_entity)
  return tot



#Politici piu nominati

In [ ]:
politici = entita_filter_by_type('Politician', entities_frame)
query ="""
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
              SELECT ?party_name
              WHERE {  
                 <TEXT_HERE>   dbp:party ?party_uri.
                 ?party_uri dbp:name ?party_name              
               }
          """
partiti = party_from_uri(entita_filter_by_type('Politician',entities_frame)['uri'], query, 'party_name')

In [80]:
politici['party'] = [x['party_name'] for x in partiti]
print('Republican Party')
display(politici[politici['party']=='Republican Party'].iloc[0:6])
print('--------------------------------------------------------------')
print('Democratic Party')
display(politici[politici['party']=='Democratic Party'].iloc[0:6])

NameError: ignored

# persone non politiche citate


In [ ]:
pd.concat([entita_filter_by_type('Person', entities_frame), politici]).drop_duplicates(subset=['label'], keep = False).iloc[0:10]


,label,count,uri,plain_text,party
115,Jervy_Cruz,115,http://dbpedia.org/resource/Jervy_Cruz,"{CRUZ, Cruz}",NaN
42,Greta_Thunberg,16,http://dbpedia.org/resource/Greta_Thunberg,"{Greta, greta}",NaN
133,Ricky_Rubio,11,http://dbpedia.org/resource/Ricky_Rubio,{Rubio},NaN
62,Pocahontas,10,http://dbpedia.org/resource/Pocahontas,{Pocahontas},NaN
26,Gabby_Barrett,9,http://dbpedia.org/resource/Gabby_Barrett,{I hope},NaN
158,Johnny_Carson,7,http://dbpedia.org/resource/Johnny_Carson,{Carson},NaN
118,Bob_Knight,6,http://dbpedia.org/resource/Bob_Knight,{Bobby Knight},NaN
130,Matt_Drudge,6,http://dbpedia.org/resource/Matt_Drudge,"{Drudge, DRUDGE}",NaN
138,Stuart_Stevens,5,http://dbpedia.org/resource/Stuart_Stevens,{Stuart Stevens},NaN
25,Phyllis_Schlafly,4,http://dbpedia.org/resource/Phyllis_Schlafly,{Phyllis Schlafly},NaN


# stati americani piu citati

In [ ]:
entita_filter_by_type('Location', entities_frame)

query ="""
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
              SELECT ?name
              WHERE {  
                 <TEXT_HERE>   dbo:country ?country.
                 ?country dbp:conventionalLongName ?name.
               }
          """
countries = party_from_uri(entita_filter_by_type('Location',entities_frame)['uri'], query, 'name')

In [ ]:
luoghi_usa = entita_filter_by_type('Location',entities_frame)
luoghi_usa['country']= [x['name'] for x in countries]
luoghi_usa[luoghi_no_usa['country'] == 'United States of America']

,label,count,uri,plain_text,country
40,Iowa,69,http://dbpedia.org/resource/Iowa,"{Iowa, IA}",United States of America
12,Ohio,56,http://dbpedia.org/resource/Ohio,"{Ohio, OHIO}",United States of America
76,New_Hampshire,48,http://dbpedia.org/resource/New_Hampshire,"{NH, New Hampshire}",United States of America
43,South_Carolina,43,http://dbpedia.org/resource/South_Carolina,"{SOUTH CAROLINA, SC, South Carolina}",United States of America
21,Florida,42,http://dbpedia.org/resource/Florida,{Florida},United States of America
...,...,...,...,...,...
149,"Costa_Mesa,_California",1,"http://dbpedia.org/resource/Costa_Mesa,_Califo...","{Costa Mesa, California}",United States of America
138,Los_Angeles,1,http://dbpedia.org/resource/Los_Angeles,{L.A.},United States of America
129,South_Dakota,1,http://dbpedia.org/resource/South_Dakota,{South Dakota},United States of America
124,"Richmond,_Virginia",1,"http://dbpedia.org/resource/Richmond,_Virginia","{Richmond, VA}",United States of America


#schieramento sugli stati (ecluso USA)


In [ ]:
luoghi = entita_filter_by_type('Country', entities_frame)
luoghi.iloc[0:10]

,label,count,uri,plain_text
0,United_States,185,http://dbpedia.org/resource/United_States,"{United States, U.S.A., American, USA, US, Uni..."
8,Mexico,21,http://dbpedia.org/resource/Mexico,{Mexico}
11,Canada,19,http://dbpedia.org/resource/Canada,"{CAN, Canada, Canadian}"
1,North_American_Free_Trade_Agreement,14,http://dbpedia.org/resource/North_American_Fre...,{NAFTA}
4,Iran,14,http://dbpedia.org/resource/Iran,"{IRAN, Iran}"
2,Russia,13,http://dbpedia.org/resource/Russia,{Russia}
9,China,11,http://dbpedia.org/resource/China,"{China, Chinese}"
18,Norway,8,http://dbpedia.org/resource/Norway,{NO}
24,United_Kingdom,6,http://dbpedia.org/resource/United_Kingdom,"{UK, U.K., Britain}"
6,Iraq,5,http://dbpedia.org/resource/Iraq,"{Iraq, IRAQ}"


In [ ]:
print(trump_entities_matrix[239,:].argmax())
print(trump_entities_matrix[:,239].argmax())
# entities_frame.iloc[535]

240
38


In [ ]:
entities_frame.iloc[240]

entita              <__main__.Entity object at 0x7f38af271ba8>
label                                               Red_carpet
count                                                        1
types                                                       []
occurrences                                              {220}
plain_text                                        {red carpet}
sentiment_mean                                            -0.5
sentiment_median                                          -0.5
Name: 240, dtype: object

In [ ]:
trump_entities_matrix[38,239]

4.0

In [ ]:
trump.iloc[1307]['text']

'Crooked Hillary Clintons foreign interventions unleashed ISIS in Syria, Iraq and Libya. She is reckless and dangerous!'

In [ ]:
entities_frame[entities_frame['label']=='China']

,entita,label,count,types,occurrences,plain_text,sentiment_mean,sentiment_median
239,<__main__.Entity object at 0x7f38aeaed630>,China,11,"[PopulatedPlace, Place, Location, Country]","{226, 1487, 1392, 1329, 1393, 3216, 1013, 1429...","{China, Chinese}",-0.137999,-0.15


# Organizzazioni

In [79]:
pd.concat([entita_filter_by_type('Organisation', entities_frame), entita_filter_by_type('PoliticalParty', entities_frame)]).drop_duplicates(subset=['label'], keep = False).drop('uri', axis=1).iloc[0:10]
# tutti le Organizzazioni che non siano un partito politico

,label,count,plain_text
2,CNN,108,"{cnn, CNN}"
19,Fox_News,97,"{Foxnews, Fox News, Fox News Channel, foxnews,..."
4,Enjoy_Records,89,"{Enjoy!, ENJOY!, Enjoy, enjoy!}"
7,United_States_Senate,54,"{senator, US Senator, Senate, Senators, Senator}"
69,Fox_Broadcasting_Company,22,"{Fox, FOX, Fox Network}"
21,People!,21,{people!}
59,Republican_National_Committee,12,"{RNC, Republican National Committee}"
1,NBC,12,{NBC}
15,WIN_Television,11,{WIN}
50,NATO,11,"{NATO, N.A.T.O}"


In [ ]:
entita_filter_by_type('Legislature', entities_frame)

,label,count,uri,plain_text
0,United_States_Senate,54,http://dbpedia.org/resource/United_States_Senate,"{US Senator, senator, Senator, Senate, Senators}"
2,United_States_House_of_Representatives,4,http://dbpedia.org/resource/United_States_Hous...,"{House, Congressman}"
1,House_of_Commons_of_the_United_Kingdom,1,http://dbpedia.org/resource/House_of_Commons_o...,{HC}
3,United_States_House_Committee_on_Rules,1,http://dbpedia.org/resource/United_States_Hous...,{Rules Committee}
4,United_States_Congress,1,http://dbpedia.org/resource/United_States_Cong...,{Congress}


In [ ]:
entita_filter_by_type('Broadcaster', entities_frame).iloc[0:5]

,label,count,uri,plain_text
2,CNN,108,http://dbpedia.org/resource/CNN,"{CNN, cnn}"
5,Fox_News,97,http://dbpedia.org/resource/Fox_News,"{foxnews, Fox News, Fox News Channel, FoxNews,..."
11,Fox_Broadcasting_Company,22,http://dbpedia.org/resource/Fox_Broadcasting_C...,"{FOX, Fox, Fox Network}"
1,NBC,12,http://dbpedia.org/resource/NBC,{NBC}
3,WILL,11,http://dbpedia.org/resource/WILL,{WILL}


In [ ]:
entita_filter_by_type('PeriodicalLiterature', entities_frame).iloc[0:5]

,label,count,uri,plain_text
3,The_New_York_Times,52,http://dbpedia.org/resource/The_New_York_Times,"{NYTimes, nytimes, NYT, NY Times}"
5,You_(South_African_magazine),40,http://dbpedia.org/resource/You_(South_African...,{YOU}
2,Look_(American_magazine),21,http://dbpedia.org/resource/Look_(American_mag...,{Look}
1,The_Wall_Street_Journal,19,http://dbpedia.org/resource/The_Wall_Street_Jo...,"{Wall Street Journal, WSJ}"
24,National_Review,12,http://dbpedia.org/resource/National_Review,"{NRO, National Review}"


#Sotituzione entità nel testo

In [ ]:
final_text = []
for index, row in trump.iterrows():
  partial = row['text']
  for ent in row['dbpedia_object'].entities:
    partial=partial.replace(ent.original_text, re.sub('[^A-Za-z0-9]+', '', ent.label))
  final_text.append(partial)


In [ ]:
# for index, row in trump.iterrows():
  # print(row['text'])
  # print(final_text[index])
  # print('---------------')

In [ ]:
pd.DataFrame(final_text, columns=['tweet']).to_csv('tweet_with_entities_trump.csv')